In [9]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import numpy as np
import portfolioGeneration
import portfolio
import dataAck
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
from IPython.display import display
import multiprocessing as mp  

In [10]:
## GET ALL MODELS


allModels = [item["model"] for item in portfolio.getModels(ticker=None, returnEntireObject=True)][:10]

for i in range(len(allModels)):
    model = allModels[i]
    print(model.describe())

((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300)
((5, ('FXE', 29, 13, None, 1), 7, 'SPY'), 5, 50)
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 150)
((5, ('USO', 19, 9, 8, 2), 15, 'IWB'), 5, 50)
((5, ('VIG', 12, None, 8, 1), 15, 'IWD'), 5, 50)
((5, ('IYR', 19, 20, 21, 3), 5, 'IAU'), 5, 50)
((10, ('USO', 19, 9, 8, 2), 15, 'IWB'), 10, 150)
((44, ('USO', 19, 9, 8, 2), 5, 'IWB'), 44, 150)
((5, ('SHY', 1, None, 5, 1), 7, 'VIG'), 5, 150)
((5, ('FXE', 29, 13, None, 1), 15, 'SPY'), 5, 100)


In [13]:
def generateAllReturnsFromCache(allModels):
    aggregateReturns = None
    aggregatePredictions = None
    aggregateSlippageReturns = None
    cleanedModels = []
    for mod in allModels:
        
        try:
            algoReturn, algoPredictions, algoSlippageAdjustedReturn = dataAck.getModelData(mod)
            print(mod.describe())
            algoReturn.columns = [str(mod.describe())]
            algoPredictions.columns = [str(mod.describe())]
            algoSlippageAdjustedReturn.columns =  [str(mod.describe())]
            if aggregateReturns is None:
                aggregateReturns = algoReturn
                aggregatePredictions = algoPredictions
                aggregateSlippageReturns = algoSlippageAdjustedReturn
            else:
                aggregateReturns = aggregateReturns.join(algoReturn)
                aggregatePredictions = aggregatePredictions.join(algoPredictions)
                aggregateSlippageReturns = aggregateSlippageReturns.join(algoSlippageAdjustedReturn)
            cleanedModels.append(mod)
        except:
            print("SKIPPING", mod.describe())
    return aggregateReturns, aggregatePredictions, aggregateSlippageReturns, cleanedModels

def computeReturnsForUniqueModelsCache(uniqueModels, factorToTrade):
    tickersRequired = []
    for mod in uniqueModels:

        print(mod.describe())
        if mod.inputSeries.targetTicker not in tickersRequired:
            tickersRequired.append(mod.inputSeries.targetTicker)
        if mod.inputSeries.series.ticker not in tickersRequired:
            tickersRequired.append(mod.inputSeries.series.ticker)
            
    if factorToTrade not in tickersRequired:
        tickersRequired.append(factorToTrade)
    
    pulledData, validTickers = dataAck.downloadTickerData(tickersRequired)

    joinedData = dataAck.joinDatasets([pulledData[ticker] for ticker in pulledData])
    
    modelReturns, modelPredictions, modelSlippageReturns, cleanedModels = generateAllReturnsFromCache(uniqueModels)
    
    return cleanedModels, modelReturns, modelPredictions, modelSlippageReturns, modelReturns.join(dataAck.getDailyFactorReturn(factorToTrade, joinedData)).dropna(), joinedData

In [14]:
factorToTrade = "SPY"
uniqueModels, modelReturns, modelPredictions, modelSlippageReturns, modelReturnsWithFactor, joinedData =\
    computeReturnsForUniqueModelsCache(allModels, factorToTrade)

((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300)
((5, ('FXE', 29, 13, None, 1), 7, 'SPY'), 5, 50)
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 150)
((5, ('USO', 19, 9, 8, 2), 15, 'IWB'), 5, 50)
((5, ('VIG', 12, None, 8, 1), 15, 'IWD'), 5, 50)
((5, ('IYR', 19, 20, 21, 3), 5, 'IAU'), 5, 50)
((10, ('USO', 19, 9, 8, 2), 15, 'IWB'), 10, 150)
((44, ('USO', 19, 9, 8, 2), 5, 'IWB'), 44, 150)
((5, ('SHY', 1, None, 5, 1), 7, 'VIG'), 5, 150)
((5, ('FXE', 29, 13, None, 1), 15, 'SPY'), 5, 100)
USO
FXE
IAU
VIG
SHY
IWD
IYR
IWB
SPY
ATTEMPTING PULL 0724944db3ce71de7023be74ae483f9fa0e50ecc3301443a85f9e7b4
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300)
ATTEMPTING PULL 0bf26d94dfb9569efaf93627fe41c8271144a095bbe91409aa8c462e
((5, ('FXE', 29, 13, None, 1), 7, 'SPY'), 5, 50)
ATTEMPTING PULL 1b2d65178501b029c40fc114ad34ad655d26939cdb0d8afe2650f394
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 150)
ATTEMPTING PULL 2a382440afde41d85f32097e403783bdaac0731aacbefd175aefd0a5
((5, ('USO', 19, 9, 8, 2), 15, 'IWB'

# ENSURE ALL MODELS HAVE PREDICTIONS IN SYSTEM AND BACKFILLED


In [15]:
from google.cloud import datastore, storage, logging
import time
import params
import hashlib
def checkAggregatePredictionsStored(model):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=params.aggregatePrediction)
            
            query.add_filter('modelHash', '=', hashlib.sha224((str(model.describe())).encode('utf-8')).hexdigest())
            retrievedPredictions = list(query.fetch(limit=1))
            if len(retrievedPredictions) > 0:
                return True
            else:
                return False
        except:
            time.sleep(10)
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))

In [16]:
def runModPredictionBackfill(mod, dataToUse, backfillDays = 30):
    ##ENSURE POPULATED FOR CORRECT PREDICTION STYLE
    pred = dataAck.computePosition([mod.makeTodayPrediction(dataToUse)])
    print(mod.describe(), pred, dataToUse.index[-1])
    portfolio.storeModelPrediction(mod, pred, dataToUse.index[-1])
    
def runBackfillMP(mod, joinedData, threadsToUse, backfillDays = 30):
    mpEngine = mp.get_context('fork')
    i = mod.inputSeries.predictionPeriod - 1 + backfillDays
    runningP = []
    while i > 0:
        while len(runningP) > threadsToUse:
            runningP = dataAck.cycleP(runningP)
        
        p = mpEngine.Process(target=runModPredictionBackfill, args=(mod, joinedData[:-i], backfillDays, ))
        p.start()
        runningP.append(p)
        i -= 1
    
    while len(runningP) > 0:
        runningP = dataAck.cycleP(runningP)
    
    print("CHECKING AGGREGATE PREDICTIONS")
    ##STORE AGGREGATE PREDICTIONS
    i = mod.inputSeries.predictionPeriod - 1 + backfillDays
    allPreds = portfolio.getPredictionsByModel(mod)
    while i > 0:
        lastDay = joinedData[:-i].index[-1]
        todayPredictions = []
        for pred in allPreds:
            ##CHECK IF PREDICTION STILL VALID
            if len(joinedData[str(pred["lastDataDayUsed"]):lastDay]) - 1 < pred["predictionLength"] and len(joinedData[str(pred["lastDataDayUsed"]):lastDay]) > 0:##GETS TRADING DAYS SINCE LAST DATA DAY
                todayPredictions.append(pred["prediction"])
       
        ##SKIP UPLOAD IF NOT ENOUGH PREDICTIONS
        print(lastDay, len(todayPredictions))
        if len(todayPredictions) == mod.inputSeries.predictionPeriod:
            pred = dataAck.computePosition(todayPredictions)
            print(mod.describe(), todayPredictions, pred)
            portfolio.storeAggregateModelPrediction(mod, pred, lastDay)
        i -= 1
    

In [17]:
for item in allModels:
    modelStatus = checkAggregatePredictionsStored(item)
    print(item.describe(), modelStatus)
    if modelStatus == False:
        ##NEED TO STORE MODEL PREDICTIONS
        portfolioGeneration.storePastPredictions([item], modelPredictions[[str(item.describe())]])
        
        ##NEED TO BACKFILL DAYS NOT CACHED
        runBackfillMP(item, joinedData, 7)

((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) False
[]
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300)
NEED TO STORE 2241
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) -1.0 2017-07-12 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-07-03 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-07-11 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-07-10 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-06-30 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-07-05 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-07-07 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-07-06 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-07-13 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-07-18 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 2017-07-14 00:00:00
((5, ('FXE', 29, 13, None, 1), 5, 'SPY'), 5, 300) 1.0 201

# PREPARE FOR AUTOMATIC GENERATOR

In [18]:
aggregateReturns, aggregatePredictions = portfolioGeneration.generateAggregateReturnsPredictions(allModels, joinedData)
cleanedReturns = aggregateReturns.dropna()
cleanedPredictions = aggregatePredictions.dropna()
hashToModel = {}
for item in allModels:
    hashToModel[portfolio.getModelHash(item)] = item

,0724944db3ce71de7023be74ae483f9fa0e50ecc3301443a85f9e7b4,0bf26d94dfb9569efaf93627fe41c8271144a095bbe91409aa8c462e,1b2d65178501b029c40fc114ad34ad655d26939cdb0d8afe2650f394,2a382440afde41d85f32097e403783bdaac0731aacbefd175aefd0a5,41f545f5052522463eebac5395473eb13b3a3d4b74d159ed78ebf518,5a677a2d61d4f219955dd820efa9894ee06148f7be83f30a28fe4423,5d2d1a5d9c4e517f3e7077c6c8cb23e62f92a7e86c0a169958507c72,6ddf72f98598480cf2a3286f3add6e33399b89d718c8ea9f0e001bb9,6e5d77d7c68327ea28044db3c78d2792af21a05e100761ef9ce26393,718b080bbcf7c87e4e2a917227fb13daff19ec8aa8026aec5e721fcc
2008-09-11,0.000289,0.000072,0.000289,NaN,NaN,0.026506,NaN,NaN,NaN,2.820494e-07
2008-09-12,-0.005948,-0.001487,-0.005948,NaN,NaN,0.023155,NaN,NaN,NaN,-5.808723e-06
2008-09-15,0.003138,0.000262,0.003138,NaN,NaN,-0.012761,NaN,NaN,NaN,3.064721e-06
2008-09-16,-0.008431,-0.000703,-0.014051,NaN,NaN,0.116987,NaN,NaN,NaN,-1.372166e-05
2008-09-17,0.005563,0.000464,0.009272,NaN,NaN,-0.027002,NaN,NaN,NaN,9.055037e-06
2008-09-18,0.007403,0.000000,0.012339,NaN,NaN,0.036281,NaN,NaN,NaN,1.927994e-05
2008-09-19,-0.001415,-0.000000,-0.007075,NaN,NaN,0.033156,NaN,NaN,NaN,-1.381801e-05
2008-09-22,0.001422,0.000711,-0.004266,NaN,NaN,-0.006284,NaN,NaN,NaN,-1.805241e-05
2008-09-23,-0.000801,-0.000301,0.000000,NaN,NaN,-0.017728,NaN,NaN,NaN,3.325888e-06
2008-09-24,-0.006842,-0.001711,-0.002932,NaN,NaN,-0.003908,NaN,NaN,NaN,2.004569e-05


In [19]:
display(cleanedReturns)

,0724944db3ce71de7023be74ae483f9fa0e50ecc3301443a85f9e7b4,0bf26d94dfb9569efaf93627fe41c8271144a095bbe91409aa8c462e,1b2d65178501b029c40fc114ad34ad655d26939cdb0d8afe2650f394,2a382440afde41d85f32097e403783bdaac0731aacbefd175aefd0a5,41f545f5052522463eebac5395473eb13b3a3d4b74d159ed78ebf518,5a677a2d61d4f219955dd820efa9894ee06148f7be83f30a28fe4423,5d2d1a5d9c4e517f3e7077c6c8cb23e62f92a7e86c0a169958507c72,6ddf72f98598480cf2a3286f3add6e33399b89d718c8ea9f0e001bb9,6e5d77d7c68327ea28044db3c78d2792af21a05e100761ef9ce26393,718b080bbcf7c87e4e2a917227fb13daff19ec8aa8026aec5e721fcc
2009-02-25,0.006098,-0.002033,0.002033,-0.002419,2.838038e-04,0.000590,-0.002073,-0.000970,0.008147,-0.005586
2009-02-26,0.012571,-0.002095,0.005587,-0.003509,4.523173e-04,0.000524,-0.003048,-0.000000,0.013302,-0.007629
2009-02-27,0.030967,-0.000704,0.022521,-0.008606,8.821805e-04,-0.000000,-0.007569,-0.002915,0.022315,-0.015272
2009-03-02,0.006099,0.000938,0.004692,-0.001513,1.751342e-04,-0.001360,-0.001348,-0.000473,0.005609,-0.002532
2009-03-03,-0.019249,-0.004442,-0.013326,0.005312,-1.756334e-04,-0.003999,0.004793,0.001540,-0.011016,0.007957
2009-03-04,0.033189,0.008935,0.028083,-0.009687,2.371144e-04,0.016841,-0.008847,-0.007839,0.020889,-0.013665
2009-03-05,-0.001417,-0.000409,-0.001199,0.000743,-1.215033e-07,0.002118,0.000686,0.000560,-0.002050,0.000581
2009-03-06,0.007345,0.002387,0.005876,-0.003406,-3.574714e-05,-0.012167,-0.003182,-0.004004,0.006415,-0.003915
2009-03-09,-0.029805,-0.012108,-0.022354,0.017520,8.545736e-04,-0.016683,0.016557,0.023204,-0.022257,0.019832
2009-03-10,-0.002849,-0.001323,-0.002442,0.001197,5.970417e-06,0.005440,0.001144,0.002238,-0.000331,0.002165


In [20]:
display(cleanedPredictions)

,0724944db3ce71de7023be74ae483f9fa0e50ecc3301443a85f9e7b4,0bf26d94dfb9569efaf93627fe41c8271144a095bbe91409aa8c462e,1b2d65178501b029c40fc114ad34ad655d26939cdb0d8afe2650f394,2a382440afde41d85f32097e403783bdaac0731aacbefd175aefd0a5,41f545f5052522463eebac5395473eb13b3a3d4b74d159ed78ebf518,5a677a2d61d4f219955dd820efa9894ee06148f7be83f30a28fe4423,5d2d1a5d9c4e517f3e7077c6c8cb23e62f92a7e86c0a169958507c72,6ddf72f98598480cf2a3286f3add6e33399b89d718c8ea9f0e001bb9,6e5d77d7c68327ea28044db3c78d2792af21a05e100761ef9ce26393,718b080bbcf7c87e4e2a917227fb13daff19ec8aa8026aec5e721fcc
2009-02-25,-0.3750,0.125000,-0.1250,0.155823,-0.020081,-0.2500,0.133545,0.0625,-0.562500,0.343506
2009-02-26,-0.5625,0.093750,-0.2500,0.169800,-0.018188,-0.1250,0.147522,0.0000,-0.640625,0.341370
2009-02-27,-0.6875,0.015625,-0.5000,0.184509,-0.015808,0.0000,0.162292,0.0625,-0.562500,0.339050
2009-03-02,-0.8125,-0.125000,-0.6250,0.199890,-0.012817,0.1250,0.177979,0.0625,-0.562500,0.337219
2009-03-03,-0.8125,-0.187500,-0.5625,0.215515,-0.009399,0.3750,0.194458,0.0625,-0.546875,0.335876
2009-03-04,-0.8125,-0.218750,-0.6875,0.231689,-0.005066,0.5000,0.211609,0.1875,-0.500000,0.334534
2009-03-05,-0.8125,-0.234375,-0.6875,0.248718,-0.000122,0.5000,0.229614,0.1875,-0.531250,0.333374
2009-03-06,-0.6250,-0.203125,-0.5000,0.265808,0.005676,0.6250,0.248352,0.3125,-0.500000,0.333130
2009-03-09,-0.5000,-0.203125,-0.3750,0.283142,0.012085,0.6250,0.267578,0.3750,-0.437500,0.332703
2009-03-10,-0.4375,-0.203125,-0.3750,0.300842,0.019165,0.5000,0.287476,0.5625,-0.468750,0.332397


In [21]:
def returnSelectAlgos(algoColumns):
    return np.random.choice(algoColumns, size=random.randint(7, len(algoColumns)), replace= False)

In [22]:
# import matplotlib.pyplot as plt
import time
import random


def performPortfolioPerformanceEstimation(thisPredictions, thisReturns, hashToModel, joinedData):
    hrpReturns, historicalWeights = portfolioGeneration.\
            produceHRPPredictions(thisReturns,\
            126, startIndex=None, maxWindowSize=False)
    print("COMPUTED HISTORICAL WEIGHTS")
    
    modelsUsed = []
    
    tickersSeen = {}
    
    for modelHash in selectedAlgorithms:
        thisModel = hashToModel[modelHash]
        modelsUsed.append(thisModel)
        print(thisModel.describe())
        if thisModel.inputSeries.targetTicker not in tickersSeen:
            tickersSeen[thisModel.inputSeries.targetTicker] = 0
        tickersSeen[thisModel.inputSeries.targetTicker] += 1
    
    ##STORE MODEL
    portfolioHash = portfolioGeneration.storePortfolio(modelsUsed,\
            description=str(tickersSeen), benchmark="SPY", portfolioType="HRP FULL")
    
    portfolioGeneration.storeHistoricalAllocations(portfolioHash, \
                    modelsUsed, historicalWeights, thisPredictions)
    
    portfolioInfo = portfolio.getPortfolioByKey(portfolioHash)
    portfolioInfo = {
        "key":portfolioInfo.key.name,
        "description":portfolioInfo["description"],
        "benchmark":portfolioInfo["benchmark"],
        "portfolioType":portfolioInfo["portfolioType"],
        "startedTrading":portfolioInfo["startedTrading"]
    }
    print(portfolioInfo)
    portfolioData = portfolioGeneration.getDataForPortfolio(portfolioHash, factorToTrade, joinedData, portfolioInfo["startedTrading"])
    portfolioGeneration.cachePortfolio(portfolioInfo, portfolioData, params.AVAILABLE_MODE)

    

In [23]:
## MP RUN            
def createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse):
    mpEngine = mp.get_context('fork')
        
    runningP = []
    while True:
        selectedAlgorithms = returnSelectAlgos(cleanedReturns.columns)
        
        while len(runningP) > threadsToUse:
            runningP = dataAck.cycleP(runningP)
        
        p = mpEngine.Process(target=performPortfolioPerformanceEstimation, args=(cleanedPredictions[selectedAlgorithms], cleanedReturns[selectedAlgorithms], hashToModel, joinedData, ))
        p.start()
        runningP.append(p)
        

In [ ]:
createPossiblePortfoliosMP(cleanedPredictions, cleanedReturns, hashToModel, joinedData, threadsToUse=4)